# 电子产品销售数据可视化

2.8 增加商品一、二级分类
2.9 用户数量

## 一、读取数据

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
import datetime
from matplotlib import font_manager
from scipy.stats import norm
import matplotlib

In [2]:
data=pd.read_csv('电子产品销售数据.csv')

## 二、数据评估及数据清理

在这一部分中，我们将对data所包含的数据进行评估，主要从两个方面进行：结构和内容，即整齐度和干净度。数据的结构性问题指不符合“每个变量为一列，每个观察值为一行，每种类型的观察单位为一个表格”这三个标准；数据的内容性问题包括存在丢失数据、重复数据、无效数据等。

In [3]:
data.sample(10)

,Unnamed: 0,event_time,order_id,product_id,category_id,category_code,brand,price,user_id,age,sex,local
215795,1765587,2020-07-29 16:49:13 UTC,2364088681637085800,1515966223509104145,2.268105e+18,electronics.smartphone,samsung,231.23,1.515916e+18,49.0,男,上海
305624,1991848,2020-08-25 06:32:48 UTC,2383347368327119388,1515966223509251343,2.268105e+18,kids.dolls,deepcool,7.38,1.515916e+18,40.0,女,北京
253186,1818903,2020-08-08 06:48:07 UTC,2371033893919785059,2273948221069852696,2.268106e+18,NaN,nika,16.18,1.515916e+18,32.0,女,四川
107923,1062455,2020-05-15 05:39:36 UTC,2353222631844479148,1515966223509104353,2.268105e+18,appliances.personal.scales,tefal,24.28,1.515916e+18,39.0,男,上海
361077,2265514,2020-09-16 09:48:44 UTC,2388440981134481452,1515966223509088640,2.268105e+18,electronics.smartphone,samsung,995.35,1.515916e+18,38.0,女,上海
409478,2457966,2020-10-05 11:15:10 UTC,2388440981134587822,1515966223509104345,2.268105e+18,appliances.personal.scales,scarlett,12.71,1.515916e+18,17.0,男,浙江
310423,2027621,2020-08-27 03:33:56 UTC,2384706893416235794,1515966223509128997,2.268105e+18,appliances.kitchen.washer,samsung,567.11,1.515916e+18,37.0,男,广东
76680,744728,2020-03-09 14:20:48 UTC,2348814674595676850,2273948218662322997,2.268106e+18,NaN,grohe,116.67,1.515916e+18,23.0,男,广东
518560,2580595,2020-11-09 05:30:03 UTC,2388440981134663092,1515966223517998565,2.268105e+18,electronics.smartphone,samsung,231.46,1.515916e+18,18.0,女,广东
451789,2505377,2020-10-17 07:41:18 UTC,2388440981134617218,1515966223510204922,2.268105e+18,furniture.bedroom.pillow,mattel,3.45,1.515916e+18,27.0,男,广东


从抽样的10行数据来看，`data`中的数据符合“每行是一个观察值，每列是一个变量”。

In [4]:
data=data[['event_time','order_id','product_id','category_id','category_code','brand','price','user_id','age','sex','local']]

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564169 entries, 0 to 564168
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   event_time     564169 non-null  object 
 1   order_id       564169 non-null  int64  
 2   product_id     564169 non-null  int64  
 3   category_id    564169 non-null  float64
 4   category_code  434799 non-null  object 
 5   brand          536945 non-null  object 
 6   price          564169 non-null  float64
 7   user_id        564169 non-null  float64
 8   age            564169 non-null  float64
 9   sex            564169 non-null  object 
 10  local          564169 non-null  object 
dtypes: float64(4), int64(2), object(5)
memory usage: 47.3+ MB


Unnamed: 行号(与分析无关) \
event_time：下单时间\
order_id：订单编号\
product_id:产品标号\
category_id :类别编号\
category_code :类别\
brand :品牌\
price :价格\
user_id :用户编号\
age :年龄\
sex :性别\
local:省份

从输出的内容来看，数据共有564169条观察值。\
其中`category_code`和`brand`变量中存在缺失值，将在后续进一步评估。\
针对数据类型，`event_time`数据类型应为“日期”，应当进行类型转换。

In [6]:
data[data['category_code'].isnull()]

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id,age,sex,local
4,2020-04-24 19:16:21 UTC,2294584263154074236,2273948316817424439,2.268105e+18,NaN,karcher,217.57,1.515916e+18,32.0,女,广东
16,2020-04-28 02:21:45 UTC,2296972701060825130,1515966223509104683,2.268105e+18,NaN,philips,23.13,1.515916e+18,21.0,男,广东
18,2020-04-28 04:25:00 UTC,2297034737199350540,1515966223509719628,2.268106e+18,NaN,NaN,6.94,1.515916e+18,37.0,女,北京
19,2020-04-28 04:25:00 UTC,2297034737199350540,1515966223509719628,2.268106e+18,NaN,NaN,6.94,1.515916e+18,37.0,女,北京
25,2020-04-29 03:25:12 UTC,2297729407910937541,1515966223509105105,2.268105e+18,NaN,sbs,0.02,1.515916e+18,47.0,男,广东
...,...,...,...,...,...,...,...,...,...,...,...
564144,2020-11-21 09:50:24 UTC,2388440981134693925,1515966223513916332,2.268105e+18,NaN,ava,78.68,1.515916e+18,44.0,男,北京
564150,2020-11-21 09:59:01 UTC,2388440981134693930,1515966223510177554,2.268105e+18,NaN,dji,0.23,1.515916e+18,31.0,女,上海
564151,2020-11-21 09:59:01 UTC,2388440981134693930,2273948237628965713,2.268105e+18,NaN,dji,0.23,1.515916e+18,31.0,女,上海
564161,2020-11-21 10:08:14 UTC,2388440981134693940,2273948246856434093,2.268105e+18,NaN,xiaomi,16.18,1.515916e+18,39.0,女,广东


由于缺失分析所需的核心数据`category_code`，我们将把这些观察值删除，并查看删除后该列空缺值个数和。

In [7]:
data=data.dropna(subset='category_code')
data['category_code'].isnull().sum()

0

In [8]:
data[data['brand'].isnull()]

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id,age,sex,local
61,2020-04-29 12:42:12 UTC,2298009757589438761,1515966223509104974,2.268105e+18,computers.peripherals.keyboard,NaN,12.71,1.515916e+18,44.0,女,广东
69,2020-04-29 12:51:50 UTC,2298014603872830244,1515966223509089420,2.268105e+18,computers.peripherals.mouse,NaN,5.07,1.515916e+18,27.0,男,重庆
88,2020-04-29 13:29:25 UTC,2298033523732251384,2273948232075706577,2.268105e+18,computers.peripherals.mouse,NaN,20.35,1.515916e+18,23.0,女,重庆
92,2020-04-29 13:31:33 UTC,2298034599193740139,2273948254808834551,2.268105e+18,appliances.kitchen.blender,NaN,74.05,1.515916e+18,40.0,女,海南
94,2020-04-29 13:31:33 UTC,2298034599193740139,1515966223509260303,2.268105e+18,furniture.kitchen.table,NaN,23.13,1.515916e+18,40.0,女,海南
...,...,...,...,...,...,...,...,...,...,...,...
563897,2020-11-21 03:49:01 UTC,2388440981134693709,2273948242175590517,2.268105e+18,computers.peripherals.mouse,NaN,2.06,1.515916e+18,25.0,女,上海
563949,2020-11-21 05:29:10 UTC,2388440981134693754,1515966223509382270,2.268105e+18,computers.components.cpu,NaN,46.27,1.515916e+18,47.0,男,北京
563963,2020-11-21 05:59:34 UTC,2388440981134693768,2337686271054316416,2.268105e+18,computers.peripherals.mouse,NaN,12.71,1.515916e+18,23.0,女,海南
563966,2020-11-21 06:03:17 UTC,2388440981134693771,2333247274437574677,2.268105e+18,furniture.kitchen.table,NaN,27.75,1.515916e+18,20.0,男,上海


由于缺失分析所需的数据`brand`对后续的分析不会产生影响，因此可以该变量的缺失值保留。

In [9]:
#因为在解析日期字符串时，时间格式和实际字符串不匹配，解析在第0个位置时遇到了一个无法解析的剩余字符串"UTC"。
data['event_time']=pd.to_datetime(data['event_time'].str[:19],format='%Y-%m-%d %H:%M:%S')
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 434799 entries, 0 to 564168
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   event_time     434799 non-null  datetime64[ns]
 1   order_id       434799 non-null  int64         
 2   product_id     434799 non-null  int64         
 3   category_id    434799 non-null  float64       
 4   category_code  434799 non-null  object        
 5   brand          420718 non-null  object        
 6   price          434799 non-null  float64       
 7   user_id        434799 non-null  float64       
 8   age            434799 non-null  float64       
 9   sex            434799 non-null  object        
 10  local          434799 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(2), object(4)
memory usage: 39.8+ MB


In [10]:
data.duplicated().sum()

525

In [11]:
data=data.drop_duplicates()
data.duplicated().sum()

0

输出结果为，说明不存在重复值。

针对`data`，不一致数据可能存在于`local`和`sex`变量中，我们将查看是否存在多个不同值指代同一地区，以及多个不同值指代同一性别的情况。

In [12]:
data['local'].value_counts()

local
广东    95097
上海    73604
北京    71075
湖南    26842
海南    26109
四川    25912
重庆    24445
浙江    23892
江苏    23288
天津    22803
湖北    21207
Name: count, dtype: int64

In [13]:
data['sex'].value_counts()

sex
男    218949
女    215325
Name: count, dtype: int64

从上面看出，`data`中并不存在不一致数据。

In [14]:
data.describe()

,event_time,order_id,product_id,category_id,price,user_id,age
count,434274,4.342740e+05,4.342740e+05,4.342740e+05,434274.000000,4.342740e+05,434274.000000
mean,2020-07-02 14:07:40.709563648,2.370498e+18,1.678239e+18,2.274194e+18,247.785934,1.515916e+18,33.152471
min,1970-01-01 00:33:40,2.294360e+18,1.515966e+18,2.268105e+18,0.000000,1.515916e+18,16.000000
25%,2020-07-02 12:06:43,2.354503e+18,1.515966e+18,2.268105e+18,37.010000,1.515916e+18,24.000000
50%,2020-08-14 09:53:42,2.375517e+18,1.515966e+18,2.268105e+18,138.870000,1.515916e+18,33.000000
75%,2020-10-07 05:26:17,2.388441e+18,1.515966e+18,2.268105e+18,328.680000,1.515916e+18,42.000000
max,2020-11-21 10:10:30,2.388441e+18,2.388434e+18,2.374499e+18,18328.680000,1.515916e+18,50.000000
std,NaN,2.014598e+16,3.180363e+17,2.419753e+16,320.248441,2.369839e+07,10.115632


从以上统计信息来看，data里不存在脱离现实意义的数值。

## 三、数据整理

为了便于后续的数据分析，将对`data`数据进行整理。

根据后续的分析，需要将下单时间的年、月、日、时间提取出来

In [15]:
data['year']=data['event_time'].dt.year
data['month']=data['event_time'].dt.month
data['day']=data['event_time'].dt.day
data['hour']=data['event_time'].dt.hour
data.sample(5)

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id,age,sex,local,year,month,day,hour
458985,2020-10-19 10:14:04,2388440981134622107,1515966223509089252,2.268105e+18,computers.peripherals.mouse,genius,5.76,1.515916e+18,41.0,女,天津,2020,10,19,10
508287,2020-11-03 13:00:45,2388440981134656367,1515966223509258228,2.268105e+18,appliances.iron,tefal,64.79,1.515916e+18,43.0,女,重庆,2020,11,3,13
311084,2020-08-27 10:08:04,2384905272276549650,1515966223509088610,2.268105e+18,electronics.smartphone,samsung,532.38,1.515916e+18,22.0,男,上海,2020,8,27,10
14207,2020-05-23 12:10:54,2315388624201319009,2273948310542746296,2.268105e+18,kids.carriage,alis,78.68,1.515916e+18,23.0,女,湖南,2020,5,23,12
491567,2020-10-28 06:38:13,2388440981134644430,2273948227654910595,2.268105e+18,appliances.environment.vacuum,samsung,104.14,1.515916e+18,50.0,女,上海,2020,10,28,6


将`data`的数据赋值给`cleaned——data`便于后续进行分析。

In [16]:
cleaned_data=data.copy()

In [17]:
#根据用户的年龄去添加用户所在的年龄段
cleaned_data['age_group']=pd.cut(cleaned_data['age'],[10,20,30,40,50],labels=['10~20','20~30','30~40','40~50'])
cleaned_data.head(5)

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id,age,sex,local,year,month,day,hour,age_group
0,2020-04-24 11:50:39,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18,24.0,女,海南,2020,4,24,11,20~30
2,2020-04-24 14:37:43,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18,38.0,女,北京,2020,4,24,14,30~40
5,2020-04-26 08:45:57,2295716521449619559,1515966223509261697,2.268105e+18,furniture.kitchen.table,maestro,39.33,1.515916e+18,20.0,男,重庆,2020,4,26,8,10~20
6,2020-04-26 09:33:47,2295740594749702229,1515966223509104892,2.268105e+18,electronics.smartphone,apple,1387.01,1.515916e+18,21.0,男,北京,2020,4,26,9,20~30
10,2020-04-26 14:55:26,2295902490203259134,2273948311742316796,2.268105e+18,appliances.kitchen.refrigerators,lg,462.94,1.515916e+18,24.0,男,北京,2020,4,26,14,20~30


In [18]:
#增加商品的一二级分类
#对列进行拆分
cleaned_data[['category_code_1','category_code_2','category_code_3']]=cleaned_data['category_code'].str.split('.',expand=True)
cleaned_data.sample(10)
cleaned_data=cleaned_data.drop('category_code_2',axis=1)

#cleaned_data=cleaned_data.rename({'category_code_3':'category_code_2'})
cleaned_data.sample(5)

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id,age,sex,local,year,month,day,hour,age_group,category_code_1,category_code_3
369171,2020-09-19 12:01:18,2388440981134504856,1515966223509089373,2.268105e+18,apparel.glove,sony,403.45,1.515916e+18,22.0,女,湖南,2020,9,19,12,20~30,apparel,None
126694,2020-06-13 10:12:38,2353262085044961612,1515966223509088560,2.268105e+18,appliances.kitchen.grill,tefal,231.46,1.515916e+18,16.0,男,上海,2020,6,13,10,10~20,appliances,grill
100322,2020-07-12 06:27:41,2351454661254316303,1515966223509089805,2.268105e+18,computers.notebook,lenovo,680.53,1.515916e+18,27.0,女,湖南,2020,7,12,6,20~30,computers,None
7148,2020-05-12 05:24:37,2307211598059537047,1515966223509089486,2.268105e+18,electronics.smartphone,samsung,115.72,1.515916e+18,35.0,女,湖北,2020,5,12,5,30~40,electronics,None
482997,2020-10-27 05:59:21,2388440981134638676,1515966223509089966,2.268105e+18,electronics.smartphone,apple,856.23,1.515916e+18,46.0,女,海南,2020,10,27,5,40~50,electronics,None


In [19]:
#用户数量
cleaned_data['user_id'].nunique()

87699

## 三、数据探索

In [20]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 434274 entries, 0 to 564168
Data columns (total 18 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   event_time       434274 non-null  datetime64[ns]
 1   order_id         434274 non-null  int64         
 2   product_id       434274 non-null  int64         
 3   category_id      434274 non-null  float64       
 4   category_code    434274 non-null  object        
 5   brand            420218 non-null  object        
 6   price            434274 non-null  float64       
 7   user_id          434274 non-null  float64       
 8   age              434274 non-null  float64       
 9   sex              434274 non-null  object        
 10  local            434274 non-null  object        
 11  year             434274 non-null  int32         
 12  month            434274 non-null  int32         
 13  day              434274 non-null  int32         
 14  hour             434274 n

In [21]:
cleaned_data.sample(5)

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id,age,sex,local,year,month,day,hour,age_group,category_code_1,category_code_3
173254,2020-07-17 10:52:26,2355211792344417006,1515966223509106943,2.268105e+18,appliances.kitchen.refrigerators,indesit,372.66,1.515916e+18,26.0,男,北京,2020,7,17,10,20~30,appliances,refrigerators
303130,2020-08-24 10:22:38,2382738271181471834,1515966223509262612,2.268105e+18,appliances.kitchen.blender,scarlett,17.34,1.515916e+18,33.0,女,海南,2020,8,24,10,30~40,appliances,blender
515677,2020-11-08 07:12:57,2388440981134661187,1515966223509284145,2.268105e+18,stationery.battery,varta,3.47,1.515916e+18,20.0,男,重庆,2020,11,8,7,10~20,stationery,None
3880,2020-05-06 19:35:31,2303291216621470336,1515966223509089401,2.268105e+18,electronics.clocks,apple,486.09,1.515916e+18,26.0,男,北京,2020,5,6,19,20~30,electronics,None
334008,2020-09-05 11:57:45,2388440981134414638,1515966223509117535,2.268105e+18,electronics.tablet,samsung,601.83,1.515916e+18,42.0,男,湖南,2020,9,5,11,40~50,electronics,None


### 3.1 每月订单数量订单额

In [ ]:
#画布大小
plt.figure(figsize=(20,15))
#显示风格
sns.set_palette('pastel')
#设置字体
font=font_manager.FontProperties(fname=r"C:\Windows\Fonts\msjhl.ttc",size=14)
#引入axes对象
fig = plt.figure()
ax1=fig.add_subplot(111)
#绘制条形图
ax1.bar(cleaned_data['month'],height=cleaned_data['price'],width=0.8,label='订单数量')
#共享X轴
ax2=ax1.twinx()
#绘制折线图
ax2.plot(cleaned_data['month'],cleaned_data['price'],marker='o',color='yellow',label='订单额')
#设置折线图节点的注释
for a,b in zip(cleaned_data['month'],cleaned_data['price']):
    plt.annotate("%.2f"%b,xy=[a,b],arrowprops=None)
#设置图表标题
ax1.set_title('2020年各月份电子产品销售订单数量图',fontproperties=font,fontsize=20)
#设置值xy轴的标题
ax1.set_xlabel('月份')
ax1.set_ylabel('订单数量')
#设置xy轴的刻度标签
ax1.set_xticks(cleaned_data['month'].astype('str'))
#设置图表的注释
fig.legend(prop=font,loc=(0.85,0.85))
plt.show()

### 3.2 一月各天订单数量分布

### 3.3 一天各时段订单数量分布

### 3.4 男女用户订单比例

### 3.5 女性/男性购买商品TOP20

### 3.6 各年龄段订单数量订单额

### 3.7 各年龄段购买商品TOP10

### 3.8 用户RFM等级画像

## 四、总结